## Data we definitely want to include:
### Outdoor polygons: OSM parks and pitches (indoor = No), Parkserve local parks and recreation.
* We selected five random cities to get
1. Bellingham, Washington — ~93,000
2. Sandpoint, Idaho — ~9,000
3. Ithaca, New York — ~32,000
4. Santa Cruz, California — ~63,000
5. Cedar Rapids, Iowa — ~138,000

In [66]:
import os
import requests
import geopandas as gpd
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import osmnx as ox
from shapely.geometry import shape


# --- CONFIG ---
PLACE_BASE_URL = "https://www2.census.gov/geo/tiger/TIGER2025/PLACE/tl_2025_{statefp}_place.zip"
CACHE_DIR = "cache/place"
os.makedirs(CACHE_DIR, exist_ok=True)

STATE_FIPS = {
    "01": "Alabama", "02": "Alaska", "04": "Arizona", "05": "Arkansas",
    "06": "California", "08": "Colorado", "09": "Connecticut", "10": "Delaware",
    "12": "Florida", "13": "Georgia", "16": "Idaho", "17": "Illinois",
    "18": "Indiana", "19": "Iowa", "20": "Kansas", "21": "Kentucky",
    "22": "Louisiana", "23": "Maine", "24": "Maryland", "25": "Massachusetts",
    "26": "Michigan", "27": "Minnesota", "28": "Mississippi", "29": "Missouri",
    "30": "Montana", "31": "Nebraska", "32": "Nevada", "33": "New Hampshire",
    "34": "New Jersey", "35": "New Mexico", "36": "New York", "37": "North Carolina",
    "38": "North Dakota", "39": "Ohio", "40": "Oklahoma", "41": "Oregon",
    "42": "Pennsylvania", "44": "Rhode Island", "45": "South Carolina",
    "46": "South Dakota", "47": "Tennessee", "48": "Texas", "49": "Utah",
    "50": "Vermont", "51": "Virginia", "53": "Washington", "54": "West Virginia",
    "55": "Wisconsin", "56": "Wyoming"
}

def load_state_places(statefp):
    """Load (or download + cache) a state's PLACE shapefile as GeoDataFrame."""
    zip_path = os.path.join(CACHE_DIR, f"tl_2025_{statefp}_place.zip")

    # Download if not already cached
    if not os.path.exists(zip_path):
        url = PLACE_BASE_URL.format(statefp=statefp)
        print(f"Downloading {STATE_FIPS[statefp]} PLACE file...")
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        with open(zip_path, "wb") as f:
            f.write(r.content)
    else:
        print(f"Using cached file for {STATE_FIPS[statefp]}")

    # Load from local zip
    return gpd.read_file(f"zip://{zip_path}")

 
def get_city_boundary(city_name, state):
    """
    Find and return the boundary GeoDataFrame for a given city within a known U.S. state.
    `state` can be the full state name (e.g., "Colorado") or FIPS (e.g., "08").
    """
    city_name = city_name.lower()

    # Resolve statefp from either state name or FIPS code
    state_lookup = {v.lower(): k for k, v in STATE_FIPS.items()}
    if state.isdigit():
        statefp = state.zfill(2)
        state_name = STATE_FIPS.get(statefp, f"FIPS {statefp}")
    else:
        statefp = state_lookup.get(state.lower())
        if not statefp:
            raise ValueError(f"Unknown state: {state}")
        state_name = state.title()

    # Load the state’s PLACE shapefile (cached if available)
    gdf = load_state_places(statefp)

    # Case-insensitive match for city name
    city_rows = gdf[gdf["NAME"].str.lower() == city_name]

    if city_rows.empty:
        raise ValueError(f"City '{city_name.title()}' not found in {state_name}.")

    city_rows = city_rows.copy()
    city_rows["state_name"] = state_name
    return city_rows




### Pull OSM data

In [230]:
# Get OSM data
tags = {
    "leisure": ["park", "pitch", "sports_centre"]
}
def load_osm(city, state):
    city_geom = get_city_boundary(city, state).geometry.iloc[0]
    
    # Download OSM features that *intersect* the city boundary
    gdf_osm = ox.features_from_polygon(city_geom, tags=tags)
    
    # Ensure correct CRS
    gdf_osm = gdf_osm.to_crs("EPSG:4326")

    if "indoor" in gdf_osm.columns:
        gdf_osm = gdf_osm.loc[gdf_osm["indoor"] != "yes"]
        print(f"number of indoor pictches/sports_centres {len(gdf_osm[gdf_osm.get("indoor") == "yes"])}")
    else:
    # No indoor column at all — nothing to filter
        pass
    gdf_osm = gdf_osm[gdf_osm.geometry.type.isin(["Polygon", "MultiPolygon"])]
    # Clip to city boundary (this is the key step!)
    gdf_osm_clipped = gdf_osm.clip(city_geom)
    #only get polygons
    return gdf_osm_clipped

### Pull ParkServe data

In [231]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import json

PARKSERVE_URL = (
    "https://server7.tplgis.org/arcgis7/rest/services/"
    "ParkServe/ParkServe_ProdNew/MapServer/2/query"
)
equal_area_crs = "EPSG:5070"  # US Albers Equal Area — ideal for area in meters²


def get_city_parks_parkserve(city_name, state_name, min_acres=0.1):
    """
    Fetch ParkServe parks that intersect a SINGLE city and return in EPSG:4326.
    
    Args:
        city_name (str): Name of the city (case-insensitive)
        state_name (str): Full state name (e.g., "Washington") or FIPS code
        min_acres (float): Minimum park size in acres to keep

    Returns:
        GeoDataFrame (EPSG:4326) ready for Folium plotting
    """
    # --- 1. Get city boundary ---
    city_row = get_city_boundary(city_name, state_name).iloc[0]  # Use your existing function
    city_geom = city_row.geometry
    city_name = city_row["NAME"]

    # Build bounding envelope for ArcGIS query
    bounds = city_geom.bounds
    envelope = {
        "xmin": bounds[0],
        "ymin": bounds[1],
        "xmax": bounds[2],
        "ymax": bounds[3],
        "spatialReference": {"wkid": 4326}
    }

    # --- 2. Fetch ParkServe features ---
    features = []
    offset = 0
    while True:
        params = {
            "where": "(park_designation = 'LP' OR park_designation = 'LREC')",
            "outFields": "*",
            "f": "geojson",
            "geometry": json.dumps(envelope),
            "geometryType": "esriGeometryEnvelope",
            "spatialRel": "esriSpatialRelIntersects",
            "resultOffset": offset,
            "resultRecordCount": 1000,
            "returnGeometry": "true",
            "outSR": 5070,  # original CRS in meters
        }
        r = requests.get(PARKSERVE_URL, params=params, timeout=60)
        r.raise_for_status()
        page = r.json().get("features", [])
        if not page: break
        features.extend(page)
        if len(page) < 1000: break
        offset += 1000

    if not features:
        print(f"No parks found for {city_name}")
        return gpd.GeoDataFrame(columns=["park_name","park_designation","geometry"], crs="EPSG:4326")

    # --- 3. Convert features to GeoDataFrame ---
    parks = gpd.GeoDataFrame(
        [f.get("attributes") or f.get("properties") for f in features if f.get("geometry")],
        geometry=[shape(f["geometry"]) for f in features if f.get("geometry")],
        crs="EPSG:5070"  # original CRS from ParkServe
    )

    # Fix invalid geometries
    parks["geometry"] = parks.geometry.buffer(0)
    parks = parks[~parks.geometry.is_empty]
    # --- 4. Clip parks to city boundary ---
    parks = parks.to_crs("EPSG:4326")
    city_gdf = gpd.GeoDataFrame(geometry=[city_geom.buffer(0)], crs="EPSG:4326")
    parks_clipped = gpd.overlay(parks, city_gdf, how="intersection")
    
    # --- 5. Filter by minimum size ---
    parks_clipped = parks_clipped.to_crs(equal_area_crs)
    parks_clipped["area_acres"] = parks_clipped.geometry.area / 4046.86
    parks_clipped = parks_clipped[parks_clipped["area_acres"] >= min_acres]
    parks_clipped = parks_clipped.to_crs("EPSG:4326")  # ready for Folium


    # --- 6. Reproject to EPSG:4326 for Folium ---
    parks_clipped = parks_clipped.to_crs("EPSG:4326")

    print(f"{city_name}: {len(parks)} parks before clip, {len(parks_clipped)} after clip")
    return parks_clipped


## Visualize data
- To Do: decide wheteher to filter data by minimum size. Some OSM/ParkServe data is very small, and likely not an intential stop. However, it would have a low impact on the final results and we would not have to justify our size cutoff if we left it in.

- #### To run, just change the city and state variables

In [232]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon

city = 'Cedar Rapids'
state = 'Iowa'

city_geom = get_city_boundary(city,state)
center = city_geom.geometry.iloc[0].centroid
center = [center.y, center.x]  

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# Add ParkServe polygons (green)
gdf = get_city_parks_parkserve(city, state)
folium.GeoJson(
    gdf,
    name="ParkServe",
    style_function=lambda x: {"color": "green", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=["ParkName"] if "ParkName" in gdf.columns else [])
).add_to(m)

# Add OSM polygons (gray)
gdf_osm = load_osm(city, state)
folium.GeoJson(
    gdf_osm,
    name="OSM Parks",
    style_function=lambda x: {"color": "gray", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=["osm_category"] if "osm_category" in gdf_osm.columns else [])
).add_to(m)

# Add layer control (toggle layers)
folium.LayerControl().add_to(m)

m.save(f"{city}_park_overlap.html")
m

Using cached file for Iowa
Using cached file for Iowa
Cedar Rapids: 114 parks before clip, 75 after clip
Using cached file for Iowa


In [79]:
gdf

,objectid,parkid,park_name,park_designation,park_local_owner,park_local_manager,park_access,park_size_acres,park_address_1,park_urbanarea,...,park_zip,park_boundary_source,park_source_agency,park_feedback_source,park_dateadded,area_sq_km,canopy_pct,trails_yn,playground_count,geometry


### Indoor polygons: OSM sports halls, fitness centres. 

- Average square footage of a fitness studio: https://member.afsfitness.com/content/fitness-studio-fact-sheet
- 3,813 ft. --> 61*61 ft

In [181]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import box
import pandas as pd

def load_osm_indoor(city, state):
    """
    Fetch OSM gyms and create ≤61×61 ft indoor zones within building footprints
    for point-only gyms, clipped by other business footprints.
    """
    # --- 1. Get city boundary ---
    
    city_geom = get_city_boundary(city, state)
    if city_geom.crs is None or city_geom.crs.to_epsg() != 4326:
        city_geom = city_geom.to_crs("EPSG:4326")
    city_geom = city_geom.geometry.iloc[0]

    # --- 2. Define OSM tags ---
    gym_tags = {"leisure": ["fitness_centre", "sports_hall"]}
    bldg_tags = {"building": True}
    business_tags = {
        "amenity": True, "shop": True, "office": True,
        "craft": True, "tourism": True, "leisure": True
    }

    # --- 3. Download all OSM features ---
    gdf_gym = ox.features_from_polygon(city_geom, tags=gym_tags)
    gdf_bldgs = ox.features_from_polygon(city_geom, tags=bldg_tags)
    gdf_biz = ox.features_from_polygon(city_geom, tags=business_tags)

    # --- 4. Separate gyms ---
    gdf_gym_points = gdf_gym[gdf_gym.geometry.geom_type == "Point"].copy()
    gdf_gym_polys = gdf_gym[gdf_gym.geometry.geom_type.isin(["Polygon", "MultiPolygon"])].copy()

    # --- 5. Project everything once (for meter units) ---
    crs_m = "EPSG:3857"
    gdf_gym_points = gdf_gym_points.to_crs(crs_m)
    gdf_gym_polys = gdf_gym_polys.to_crs(crs_m)
    gdf_bldgs = gdf_bldgs.to_crs(crs_m)
    gdf_biz = gdf_biz.to_crs(crs_m)

    gdf_bldgs = gdf_bldgs.reset_index(drop=True)

    # --- 6. Spatial joins: assign buildings ---
    gyms_in_bldg = gpd.sjoin(gdf_gym_points, gdf_bldgs[["geometry"]], how="inner", predicate="within")
    gyms_in_bldg.rename(columns={"index_right": "building_id"}, inplace=True)

    biz_in_bldg = gpd.sjoin(gdf_biz, gdf_bldgs[["geometry"]], how="inner", predicate="within")

    # Handle dynamic naming of the building index column
    join_col = None
    for col in biz_in_bldg.columns:
        if col.startswith("index_") or col.endswith("_right"):
            join_col = col
            break
    
    if join_col is None:
        raise KeyError("Could not find building index column in business join result")
    
    # Group businesses by building ID
    biz_in_bldg.rename(columns={join_col: "building_id"}, inplace=True)
    biz_grouped = biz_in_bldg.groupby("building_id")

    # --- 7. Create indoor zones for point-based gyms ---
    zones = []
    side_m = 18.6  # 61 ft
    half = side_m / 2
    
    for idx, row in gyms_in_bldg.iterrows():
        pt = row.geometry
        bldg_geom = gdf_bldgs.loc[row["building_id"]].geometry
    
        # Create 61x61 ft square, clipped to building
        x, y = pt.x, pt.y
        square = box(x - half, y - half, x + half, y + half)
        zone = square.intersection(bldg_geom)
    
        # Clip by nearby businesses in same building
        if row["building_id"] in biz_grouped.groups:
            biz_points = biz_grouped.get_group(row["building_id"]).geometry
            if not biz_points.empty:
                min_dist = biz_points.distance(pt).min()
                if pd.notna(min_dist) and min_dist > 0:
                    half_dist = min(min_dist / 2.0, half)
                    zone = zone.intersection(pt.buffer(half_dist)).intersection(bldg_geom)
    
        # Preserve attributes from the original point
        zones.append({
            "geometry": zone,
            "gym_name": row.get("name", None),
            "leisure": row.get("leisure", None),
            "amenity": row.get("amenity", None),
            "building_id": row["building_id"]
        })
    
    gdf_zones = gpd.GeoDataFrame(zones, crs=crs_m)


    # --- 8. Combine with polygon gyms (already defined interiors) ---
    all_gyms = pd.concat([gdf_zones, gdf_gym_polys], ignore_index=True)
    all_gyms = all_gyms.set_crs(crs_m, allow_override=True).to_crs("EPSG:4326")

    # --- 9. Clip to city boundary ---
    all_gyms = all_gyms.clip(city_geom)

    return all_gyms


### To Run, just change the city and state variables

In [183]:
city = 'Bellingham'
state = 'Washington'

city_geom = get_city_boundary(city,state)
center = city_geom.geometry.iloc[0].centroid
center = [center.y, center.x]  

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# Add OSM polygons (gray)
gdf_osm = load_osm_indoor(city, state)
folium.GeoJson(
    gdf_osm,
    name="OSM Parks",
    style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=[c for c in ["name", "leisure", "landuse", "osm_category"] if c in gdf_osm.columns])
).add_to(m)

# Add layer control (toggle layers)
folium.LayerControl().add_to(m)

m.save(f"{city}_indoor.html")
m

Using cached file for Washington
Using cached file for Washington


## Possible Additions: 

Trailheads and indoor play areas

Suggestion: Do not include. There are only 2,000 tagged indoor play areas in the entire world, and over 35,000 places in the US. It is not a large amount of data points to include, especially with the likely small area and focus on children (who don't typically carry phones while playing). Hiking is somewhat more feasible, however, they would need to be grabbed by county instead of city, and they are unevently distributed compared to trails. Moreover, the number of trails in the county depends a lot more on geography compared to city level parks.

In [222]:
# Get OSM data
# --- 2. Define OSM tags ---
recreation_tags = {
    "leisure": [
        "indoor_play",
        "trailhead"
    ],
    "information": ["trailhead"],
    "highway": ["trailhead"]
}

def load_osm_uncertain(city, state):
    city_geom = get_city_boundary(city, state).geometry.iloc[0]
    
    # Download OSM features that *intersect* the city boundary
    gdf_osm = ox.features_from(city+","+state, tags=recreation_tags)
    
    # Ensure correct CRS
    gdf_osm = gdf_osm.to_crs("EPSG:4326")
    #gdf_osm = gdf_osm[gdf_osm.geometry.type.isin(["Polygon", "MultiPolygon"])]
    # Clip to city boundary (this is the key step!)
    #gdf_osm_clipped = gdf_osm.clip(city_geom)
    #only get polygons
    return gdf_osm

In [233]:
city = 'Linn County'
state = 'Iowa'

city_geom = get_city_boundary('Cedar Rapids', 'Iowa')
center = city_geom.geometry.iloc[0].centroid
center = [center.y, center.x]  

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# Add OSM polygons (gray)
gdf_osm = load_osm_uncertain(city, state)
folium.GeoJson(
    gdf_osm,
    name="OSM Parks",
    style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=[c for c in ["name", "leisure", "landuse", "osm_category", "highway", "information"] if c in gdf_osm.columns])
).add_to(m)

# Add layer control (toggle layers)
folium.LayerControl().add_to(m)

m.save(f"{city}_indoor.html")
m

Using cached file for Iowa
